# Intro: ELPI for logic in GLIF

This notebook briefly introduces ELPI and shows how to use it in GLIF/Jupyter.
As an example, we will develop some ELPI code for propositional logic.

## Jupyter/GLIF Intro

Jupyter is a platform for developing notebooks like this one.
Jupyter is most often used for Python, but this notebook uses the [GLIF kernel](https://github.com/jfschaefer/glifkernel), which combines the systems GF, MMT and ELPI into a framework for experimenting with natural language semantics.

A notebook is a sequence of **cells**.
**Markdown cells** can be used for documentation. This text, for example, is written in a markdown cell.
**Code cells** are used for code.
You can run cells by clicking on them and then pressing `Shift + Return`.

Here is an example code cell that uses GLIF's `put_string` command:

In [1]:
put_string "Hello world"

Hello world

## Syntax of propositional logic in ELPI

Let us specify the syntax of propositional logic in ELPI.
We start by introducing a type for propositions. `o` and `prop` are common names for the type of propositions, but they are already used by ELPI, so we will use `oo` instead.

Then we can define logical connectives as functions. For example, `and` will take two propositions as arguments and give us a new proposition.
For propositional variables, we make a function `pvar` with type `int -> oo`, which gives us infinitely many variables (e.g. `pvar 13`). Note that these are variables in the logic we are defining, *not* ELPI/Prolog variables.

In [2]:
elpi: proplogsyntax
% The line above is needed for the Jupyter kernel to know that ELPI code follows.
% `proplogsyntax` will be the file name.

kind oo type.
type neg oo -> oo.
type and oo -> oo -> oo.
type or oo -> oo -> oo.
type impl oo -> oo -> oo.
type true oo.
type false oo.
type pvar int -> oo.

Successfully imported proplogsyntax.elpi proplogsyntax.elpi is the new default file for ELPI commands

## A first predicate: `is_atomic`

Computation in Prolog/ELPI is based on predicates.
As a very simple example, we will make a predicate `is_atomic`.
Atomic propositions are propositions without connectives.
In our example, that would be the variables and `true` and `false`.

A predicate is a function that returns a truth value. In this case, the predicate requires one argument of type `oo`. Here is the corresponding ELPI code:

In [3]:
elpi: atomic_first_attempt
accumulate proplogsyntax.   % import everything from our definition of the syntax

type is_atomic oo -> prop.

Successfully imported atomic_first_attempt.elpi atomic_first_attempt.elpi is the new default file for ELPI commands

Now we can test the predicate in a query using GLIF's `query` command:

In [4]:
query "is_atomic (pvar 5)"

Query failed

The query has failed, which essentially means that according to our specification, `pvar 5` is not atomic.
Of course, we haven't really told ELPI what "atomic" means.
As ELPI has the **closed-world assumption**, it assumes that anything that it cannot prove to be true (like `is_atomic (pvar 5)` in this case) is assumed to be false.

Let's fix that:

In [5]:
elpi: atomic_second_attempt
accumulate proplogsyntax.   % import everything from our definition of the syntax

type is_atomic oo -> prop.
is_atomic false.
is_atomic true.
is_atomic (pvar _X).

Successfully imported atomic_second_attempt.elpi atomic_second_attempt.elpi is the new default file for ELPI commands

We tell ELPI that `is_atomic` should succeed for `false` and `true`, and for any propositional variables.
ELPI considers identifiers that start with an uppercase letter as variables that can be unified.
So `pvar X` can be unified with `pvar 5` by setting `X = 5`.

In this case, we used `_X` instead of `X` to indicate that this is a throw-away variable (otherwise, ELPI complains).

Let's try it:

In [6]:
query "is_atomic true"

Query succeeded

In [7]:
query "is_atomic (pvar 32)"

Query succeeded

In [8]:
# this will fail because of the closed-world assumption
query "is_atomic (and true true)"

Query failed

## Task: `is_complex`

Implement a predicate `is_complex`.
A proposition is complex iff it contains connectives (i.e. the oppositive of atomic).

## A more useful predicate: `geval`

Let us now develop a more interesting predicate: `geval`.
`geval` is supposed to evaluate propositional formulae.
For example, `(and true false)` should evaluate to `false` and `(or (neg false) false)` should evaluate to `true`.
For now, we will only consider **ground formulae**, i.e. formulae without any variables.

We could implement `geval` as a unary predicate, i.e. `geval (and true false)` should fail, for example.
However, we will instead implement it as a binary predicate:
The first argument is the formula to be evaluated and the second argument is the result of the evaluation.
So `geval (and true false) false` should succeed and `geval (and true true) false` should fail.
In other words, `geval A B` succeeds iff `B` is the correct evaluation of `A`.
The advantage of this approach is that it easily generalizes e.g. to three-valued logic (where we might have `true`, `false` and `unknown`).
We will see that we can exploit the ELPI's unification to compute get something like the "input"/"output" behaviour we know from functions in functional programming.

In [9]:
elpi: geval
accumulate proplogsyntax.

type geval oo -> oo -> prop.

% `true` evaluates to `true`
geval true true.
% `false` evaluates to `false`
geval false false.

% X∧Y evaluates to true if:
%  * X evaluates to true, and
%  * Y evaluates to true.
geval (and X Y) true :- geval X true, geval Y true.
% We can think of `:-` as "if" and of `,` as "and".

% X∧Y evaluates to false if X evaluates to false
geval (and X _Y) false :- geval X false.
% X∧Y evaluates to false if Y evaluates to false
geval (and _X Y) false :- geval Y false.

% negation
geval (neg X) true :- geval X false.
geval (neg X) false :- geval X true.

% TASK: add support for `or` and `impl`

Successfully imported geval.elpi geval.elpi is the new default file for ELPI commands

Let's try it out:

In [10]:
query "geval (and true false) true"
query "geval (and true false) false"

Query failed

Query succeeded

A much more elegant way to use `geval` is to use an ELPI variable for the second argument.
ELPI then tries to find values for it that make the query successful:

In [11]:
query "geval (and true false) true"

Query failed

Of course, nothing prevents us from using a ELPI variables in other places.
For example:

In [12]:
query "geval (and (neg false) true) R"

Query succeeded R = true

## Task: `is_ground`

As mentioned above, `geval` only works for ground formulae, i.e. formulae without propositional variables.

1. What happens if you call it on a formulae with a propositional variable?
2. Implement a predicate `is_ground` that succeeds iff its argument is a ground propositional formula.

## Simplifier

As you may have noticed (if you did the tasks): It can get annoying to deal with a large number of connectives. We only have `and`, `neg`, `or` and `impl`, but it would be easy to extend it to even more connectives (`xor`, `nand`, `equiv`, ...).

Logicians typically deal with this by restricting themselves to a minimal set of connectives and thinking of the others as syntactic sugar.
For example, we can restrict ourselves to `and` and `neg`.
Then `and X Y` would be syntactic sugar for `neg (or (neg X) (neg Y))`.

In this section, we implement a simplifier that replaces `or` and `impl` in a propositional formula so that we are only left with `neg` and `and`.
Like with `geval`, we will implement this as a binary predicate and will use the second argument as an "output" by querying it with a variable for the second argument.

In [13]:
elpi: simplifier
accumulate proplogsyntax.
accumulate atomic_second_attempt.

type simplify oo -> oo -> prop.
% if `X` is atomic, then `X` simplifies to `X`.
simplify X X :- is_atomic X.
% if `X` simplifies to `X2`, then `neg X` simplfies to `neg X2`
simplify (neg X) (neg X2) :- simplify X X2.
% if `X` simplifies to `X2` and `Y` simplifies to `Y2`, then `and X Y` simplifies to `and X2 Y2`
simplify (and X Y) (and X2 Y2) :-
    simplify X X2, simplify Y Y2.
simplify (or X Y) (neg (and (neg X2) (neg Y2))) :-
    simplify X X2, simplify Y Y2.
simplify (impl X Y) (neg (and X2 (neg Y2))) :-
    simplify X X2, simplify Y Y2.
% we also could have done:
% simplify (impl X Y) Z :- simplify (or (neg X) Y) Z.

Successfully imported simplifier.elpi simplifier.elpi is the new default file for ELPI commands

In [14]:
query "simplify (impl (neg (pvar 1)) true) R"

Query succeeded R = neg and (neg pvar 1) (neg true)

## Side note: Lists in ELPI

We could make our own data type for lists (like we did for propositional formulae),
but ELPI already has built-in support for lists.

We can write an ELPI list like this: `[1, 2, 3]`

For matching, we can de-compose a list into its head (the first element) and its tail (the rest). For example, `[H|T]` matches `[1, 2, 3]` with `H=2` and `T=[2, 2]`.

As an example, we will implement an `ismember` where `ismember X L` succeeds iff `X` is an element of the list `L`.

In [15]:
elpi: listutils

type ismember T -> list T -> prop.
% If the list starts with X (i.e. X is the head), then X is a member of the list
ismember X [X|_T].
% If X is a member of T, then X is also a member of the list [_H|T], i.e. a list with one more element in front.
ismember X [_H|T] :- ismember X T.

% If you have a functional programming background, you might have expected something like
%    ismember X [] = false
%    ismember X [X|_T] = true
%    ismember X [_Y|T] = ismember X T
% ELPI/Prolog works differently because of the closed-world assumption.
% For example, in our case "ismember 1 []" fails because there is no rule that makes it work.

Successfully imported listutils.elpi listutils.elpi is the new default file for ELPI commands

In [16]:
query "ismember 2 [1,2,3]"

Query succeeded

In [17]:
query "ismember 5 [1,2,3]"

Query failed

## Finale: `beval`, a better evaluator

Our previous evaluator, `geval`, did not support propositional variables.
We will finish this notebook by implementing a better evaluator (`beval`) that also supports propositional variables.

For this, we will need to think about variable assignments.
For example, to evaluate `and (pvar 3) true`, we need to now if variable 3 is `true` or `false`.
In the following code, we will create a type `assignment` for such assignments.
We will use `asgn 3 true` to indicate that variable 3 is true.

`beval` is now a ternary predicate. The arguments are:
1. A list of variable assignments
2. A propositional formula
3. The result of the evaluation

In [18]:
elpi: beval
% a better evaluator (supports variables via assignments)
accumulate proplogsyntax.
accumulate listutils.
accumulate simplifier.

kind assignment type.
type asgn int -> oo -> assignment.

type beval (list assignment) -> oo -> oo -> prop.
% true evaluates to true
beval _Phi true true.
% false evaluates to false
beval _Phi false false.
% if variable `X` is assigned the value `Y`, then `pvar X` evaluates to `Y`.
beval Phi (pvar X) Y :- ismember (asgn X Y) Phi.

% the rest is basically copied from geval, but we pass the variable assignment around
beval Phi (and X Y) true :- beval Phi X true, beval Phi Y true.
beval Phi (and X _Y) false :- beval Phi X false.
beval Phi (and _X Y) false :- beval Phi Y false.
beval Phi (neg X) true :- beval Phi X false.
beval Phi (neg X) false :- beval Phi X true.


% Let's make a predicate that first calls the simplifier and then `beval`:
type myeval (list assignment) -> oo -> oo -> prop.
myeval Phi X Y :- simplify X X2, beval Phi X2 Y.

Successfully imported beval.elpi beval.elpi is the new default file for ELPI commands

In [19]:
query "myeval [asgn 3 true, asgn 5 false] (impl (pvar 3) (neg (pvar 5))) R"

Query succeeded R = true

In [20]:
query "myeval [asgn 3 true, asgn 5 true] (impl (pvar 3) (neg (pvar 5))) R"

Query succeeded R = false